In [5]:
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import plotly.graph_objects as go
import numpy as np
pd.set_option('display.max_rows', 500)

In [6]:
# happy_data = pd.read_csv('happiness_suicide_dataset.csv')
# Happiness rating, HDI, GDP per capita, Gini, Life expectancy, Region, Population, Household debt % of GDP, Median Age

data = pd.read_csv('master.csv')
gini = pd.read_csv('Gini coef.txt', sep='\t', header=None)[[1,2]]
gini.rename(columns={1: "country", 2: "gini"}, inplace=True)
HDI = pd.read_csv('Human Development Index (HDI).csv', encoding='latin-1')[['Country', '2015']]
HDI.rename(columns={"Country": "country", "2015": "HDI"}, inplace=True)
life_ex = pd.read_csv('Life expectancy.csv', sep=',')[['Country Name', '2015']]
life_ex.rename(columns={"Country Name": "country", "2015": "life_expectancy"}, inplace=True)
debt = pd.read_csv('Household debt % of GDP.csv', sep=',')[['Country', '2015']]
debt.rename(columns={"Country": "country", "2015": "household_debt_percent_GDP"}, inplace=True)
debt['country'] = debt['country'].apply(lambda x: x[1:])
median_age = pd.read_csv('Median age by country.csv', sep=',', header=None)[[0, 1]]
median_age.rename(columns={0: "country", 1: "median_age"}, inplace=True)
happiness = pd.read_csv("Happiness_2015.csv")[['Country', 'Happiness Score']]
happiness.rename(columns={"Country": "country", "Happiness Score": "happiness"}, inplace=True)


def clean_country(country):
    if country[:7]=='uropean':
        return country[14:]
    while country[0] == '\xa0':
        country = country[1:]
    else: 
        return country    
debt['country'] = debt['country'].apply(lambda x: clean_country(x))


In [7]:

d ={"Republic of the Congo" : "Congo", 
    "Korea(Republic of)" : "South Korea", 
    "Republic of Korea" : "South Korea", 
    "Russian Federation" : "Russia",
    "Dem. Rep. Congo" : "Congo. Dem",
    "Côte d'Ivoire" : "Ivory Coast",
    "Korea" : "South Korea",
    "Bolivia (Plurinational State of)" : "Bolivia",
    "Congo (Democratic Republic of the)" : "Congo. Dem",
    "Iran (Islamic Republic of)" : "Iran",
    "Korea (Republic of)" : "South Korea",
    "Lao People's Democratic Republic" : "Laos",
    "Moldova (Republic of)" : "Moldova",
    "Tanzania (United Republic of)" : "Tanzania",
    "Venezuela (Bolivarian Republic of)" : "Venezuela",
    "Viet Nam" : "Vietnam",
    "Korea (Democratic People's Rep. of)" : "North Korea",
    "Korea (South)" : "South Korea",
    "United Kingdom of Great Britain and Northern Ireland" : "United Kingdom",
    "United States of America" : "United States",
    "Korea (North)" : "North Korea",
    "Myanmar (Burma)" : "Burma",
    "Côte d'Ivoire (Ivory Coast)" : "Ivory Coast",
    "Congo (Republic)" : "Congo",
    "Congo (Democratic Republic)" : "Congo. Dem",
    "Bahamas, The" : "Bahamas",
    "Cote d'Ivoire" : "Ivory Coast",
    "Congo, Dem. Rep." : "Congo. Dem",
    "Congo, Rep." : "Congo",
    "Egypt, Arab Rep." : "Egypt", 
    "Hong Kong SAR, China" : "Hong Kong",
    "Iran, Islamic Rep" : "Iran",
    "Korea, Rep" : "South Korea",
    "Korea, Dem. People’s Rep." : "North Korea",
    "Venezuela, RB" : "Venezuela",
    "Yemen, Rep." : "Yemen",
    "Iran, Islamic Rep." : "Iran",
    "Korea, Rep." : "South Korea",
    "Russian" : "Russia",
    "Slovak Republic" : "Slovakia",
    "Somaliland region" : "Somalia",
    "Syrian Arab Republic" : "Syria",
    "Virgin Islands (U.S.)" : "Virgin Islands",
    "Virgin Islands of the U.S." : "Virgin Islands",
    "Czechia" : "Czech Republic",
    "Eswatini (Kingdom of)" : "Eswatini",
    "Hong Kong, China (SAR)" : "Hong Kong", 
    "Lao PDR" : "Laos",
    "Micronesia (Federated States of)" : "Micronesia",
    "Micronesia, Fed. Sts." : "Micronesia",
    "Saint Vincent and the Grenadines" : "Saint Vincent and Grenadines",
    "Sint Maarten (Dutch part)" : "Sint Maarten",
    
    }

gini.replace({"country": d}, inplace=True)
HDI.replace({"country": d}, inplace=True)
life_ex.replace({"country": d}, inplace=True)
debt.replace({"country": d}, inplace=True)
median_age.replace({"country": d}, inplace=True)
data.replace({"country": d}, inplace=True)

In [8]:
print('dimenstion of gini: ',  gini.shape)
print('dimenstion of HDI: ',  HDI.shape)
print('dimenstion of life_ex: ',  life_ex.shape)
print('dimenstion of debt: ',  debt.shape)
print('dimenstion of median_age: ',  median_age.shape)

dimenstion of gini:  (158, 2)
dimenstion of HDI:  (197, 2)
dimenstion of life_ex:  (264, 2)
dimenstion of debt:  (83, 2)
dimenstion of median_age:  (228, 2)


In [9]:
data.head()

,country,region,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,EASTERN EUROPE,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,EASTERN EUROPE,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,EASTERN EUROPE,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,EASTERN EUROPE,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,EASTERN EUROPE,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [10]:
# master_grouped = data.groupby(['country', 'year', 'region', 'gdp_per_capita ($)'], as_index=False).sum()
# master_grouped['suicides/100k pop'] = master_grouped[['suicides_no', 'population']].apply(lambda  x:x[0]/x[1]*100000, axis=1)
# master_grouped.drop(columns=['suicides_no'],inplace=True)
# master_grouped.rename(columns={"gdp_per_capita ($)" : "GDP per capita usd"}, inplace=True)
# master_grouped = master_grouped[master_grouped['year']>2010]
# master_grouped.head()

In [14]:
master2015 = data[data['year']==2015]
master_grouped = master2015.groupby(['country', 'region', 'gdp_per_capita ($)'], as_index=False).sum()
master_grouped['suicides/100k pop'] = master_grouped[['suicides_no', 'population']].apply(lambda  x:x[0]/x[1]*100000, axis=1)
master_grouped.drop(columns=['suicides_no'],inplace=True)
master_grouped.rename(columns={"gdp_per_capita ($)" : "GDP per capita usd"}, inplace=True)
master_grouped.head()

,country,region,GDP per capita usd,year,population,suicides/100k pop,HDI for year
0,Antigua and Barbuda,LATIN AMER. & CARIB,14853,24180,91889,1.088270,0.0
1,Argentina,LATIN AMER. & CARIB,14981,24180,39699624,7.740627,0.0
2,Armenia,CENTRAL ASIA,3775,24180,2795335,2.647268,0.0
3,Australia,OCEANIA,60656,24180,22240785,13.610131,0.0
4,Austria,WESTERN EUROPE,46484,24180,8219386,15.220115,0.0


In [94]:
# master_grouped.to_csv('suicide data post 2010.csv', index=False)

In [10]:
merged = master_grouped.merge(gini, on='country', how='outer')
merged = merged.merge(HDI, on='country', how='outer')
merged = merged.merge(median_age, on='country', how='outer')
merged = merged.merge(life_ex, on='country', how='outer')
merged = merged.merge(debt, on='country', how='outer')
merged = merged.merge(happiness, on='country', how='outer')
merged.dropna(axis=0, how='all', thresh=None, subset=None, inplace=True)

In [11]:
display(merged['country'].nunique())
# merged['country'].unique()

298

In [54]:
display(merged)
merged.shape

,country,region,GDP per capita usd,population,suicides/100k pop,gini,HDI,median_age,life_expectancy,household_debt_percent_GDP,happiness
0,Antigua and Barbuda,LATIN AMER. & CARIB,14853.0,91889.0,1.088270,NaN,0.775,31.6,76.483000,NaN,NaN
1,Argentina,LATIN AMER. & CARIB,14981.0,39699624.0,7.740627,42.7,0.822,31.5,76.068000,6.35,6.574
2,Armenia,CENTRAL ASIA,3775.0,2795335.0,2.647268,32.4,0.748,34.6,74.467000,NaN,4.350
3,Australia,OCEANIA,60656.0,22240785.0,13.610131,34.7,0.936,38.6,82.400000,120.14,7.284
4,Austria,WESTERN EUROPE,46484.0,8219386.0,15.220115,30.5,0.903,43.8,81.190244,50.21,7.200
5,Belgium,WESTERN EUROPE,42830.0,10624356.0,17.572830,28.1,0.913,41.4,80.992683,59.24,6.937
6,Belize,LATIN AMER. & CARIB,5561.0,319835.0,8.129192,53.3,0.709,22.4,74.034000,NaN,NaN
7,Brazil,LATIN AMER. & CARIB,9431.0,191087085.0,5.841839,51.3,0.757,31.6,74.994000,24.44,6.983
8,Chile,LATIN AMER. & CARIB,14729.0,16565725.0,11.095198,47.7,0.84,34.0,79.646000,40.92,6.670
9,Colombia,LATIN AMER. & CARIB,6552.0,44490427.0,5.241577,51.1,0.742,29.6,76.531000,25.26,6.477


(298, 11)

In [62]:
# merged.to_csv('merged.csv', index=False)

In [ ]:
# Impute missing values
# 1. Data from other years
# 2. Data from of similar countries
# 3. Mean/Median